In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/turkish-news-dataset/new_data.csv
/kaggle/input/turkish-news-dataset/cleaned.csv
/kaggle/input/turkish-news-dataset/glove.twitter.27B.100d.txt
/kaggle/input/turkish-news-dataset/stop-words.txt


In [5]:
!pip install -Uqq transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 100.9 MB/s eta 0:00:0000:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 81.5 MB/s eta 0:00:00:00:01


In [ ]:
import os
import re
import time
import emoji
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, precision_score, recall_score, f1_score
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.metrics import CategoricalAccuracy, Precision, Recall
from collections import Counter
from datetime import datetime
import csv
import gc

from transformers import AutoTokenizer, TFAutoModel
import warnings
warnings.filterwarnings('ignore')

# BELLEK OPTİMİZASYONU: Mixed Precision kullan
from tensorflow.keras import mixed_precision
policy = mixed_precision.Policy('mixed_float16')
mixed_precision.set_global_policy(policy)
print(f'Compute dtype: {policy.compute_dtype}')
print(f'Variable dtype: {policy.variable_dtype}')

# GPU kontrol ve bellek ayarları
def check_gpu():
    gpus = tf.config.experimental.list_physical_devices('GPU')
    if gpus:
        print(f"{len(gpus)} GPU bulundu:")
        for gpu in gpus:
            print(f"  {gpu}")
        for gpu in gpus:
            try:
                # BELLEK OPTİMİZASYONU: Memory growth + limit
                tf.config.experimental.set_memory_growth(gpu, True)
                # İsteğe bağlı: GPU belleğini sınırla (örn. 8GB)
                # tf.config.set_logical_device_configuration(
                #     gpu,
                #     [tf.config.LogicalDeviceConfiguration(memory_limit=8192)]
                # )
                print(f"GPU bellek büyümesi {gpu} için etkinleştirildi")
            except RuntimeError as e:
                print(f"GPU bellek büyümesi {gpu} için ayarlanamadı: {e}")
        return True
    else:
        print("GPU bulunamadı, işlemler CPU üzerinde gerçekleştirilecek")
        return False

# F1 metriği
class F1Score(tf.keras.metrics.Metric):
    def __init__(self, name='f1_score', **kwargs):
        super(F1Score, self).__init__(name=name, **kwargs)
        self.precision = tf.keras.metrics.Precision()
        self.recall = tf.keras.metrics.Recall()

    def update_state(self, y_true, y_pred, sample_weight=None):
        y_pred = tf.argmax(y_pred, axis=1)
        y_true = tf.argmax(y_true, axis=1)
        self.precision.update_state(y_true, y_pred, sample_weight)
        self.recall.update_state(y_true, y_pred, sample_weight)

    def reset_state(self):
        self.precision.reset_state()
        self.recall.reset_state()

    def result(self):
        p = self.precision.result()
        r = self.recall.result()
        return 2 * ((p * r) / (p + r + tf.keras.backend.epsilon()))

# Türkçe stopword temizliği
def load_turkish_stopwords(stopwords_path):
    stoplist = []
    try:
        with open(stopwords_path, "r", encoding='utf-8') as f:
            for line in f:
                if line.strip() and line.strip()[0:1] != "#":
                    for word in line.split():
                        stoplist.append(word)
    except Exception as e:
        print(f"Stopword dosyası yüklenirken hata: {e}")
    return stoplist

# Metin temizleme fonksiyonu
def clean_turkish_text(text, stoplist=None):
    text = text.lower()
    text = re.sub("\n", " ", text)
    text = re.sub(r'http\S+', '', text)
    text = re.sub(r'www\S+', '', text)
    text = re.sub(r'\S+@\S+', '', text)
    text = emoji.replace_emoji(text)
    text = re.sub(r'<.*?>', '', text)
    text = re.sub(r'[0-9]+', '', text)
    text = re.sub(r'[^\w\s]', ' ', text)
    
    if stoplist:
        words = text.split()
        text = ' '.join([word for word in words if word not in stoplist])
    
    text = ' '.join(text.split())
    return text

def create_directories():
    dirs = [
        "outputs",
        "outputs/models",
        "outputs/history",
        "outputs/images",
        "outputs/confusion_matrices"
    ]
    for dir_path in dirs:
        os.makedirs(dir_path, exist_ok=True)
    print("Dizin yapısı hazırlandı")

# BELLEK OPTİMİZASYONU: Veri setini küçük parçalarda yükle
def load_and_preprocess_data(data_path, text_column="Haber Gövdesi Cleaned", 
                             label_column="Sınıf", stopwords_path=None, 
                             max_length=128, sample_size=None):  # max_length 256->128
    """Veri setini yükler ve ön işler"""
    print(f"Veri seti yükleniyor: {data_path}")
    
    try:
        # BELLEK OPTİMİZASYONU: Sadece gerekli sütunları yükle
        df = pd.read_csv(data_path, usecols=[text_column, label_column], encoding='utf-8')
        
        # BELLEK OPTİMİZASYONU: Test için küçük sample
        if sample_size:
            print(f"Veri seti {sample_size} örneğe düşürülüyor (test için)")
            df = df.sample(n=min(sample_size, len(df)), random_state=42)
        
        df = df[[text_column, label_column]].copy()
        df.columns = ['text', 'label']
    except Exception as e:
        raise ValueError(f"Veri yüklenirken hata oluştu: {str(e)}")
    
    label_encoder = LabelEncoder()
    y_encoded = label_encoder.fit_transform(df['label'])
    num_classes = len(label_encoder.classes_)
    
    print(f"Sınıf sayısı: {num_classes}")
    print(f"Sınıflar: {label_encoder.classes_}")
    
    X = df['text']
    print(f'NaN sayısı: {X.isna().sum()}')
    X = X.dropna()
    y_encoded = y_encoded[X.index]
    
    X_train, X_test, y_train, y_test = train_test_split(
        X, y_encoded, test_size=0.1, random_state=42, stratify=y_encoded
    )

    X_train = X_train.reset_index(drop=True)
    X_test = X_test.reset_index(drop=True)
    
    # One-hot encoding
    y_train_onehot = tf.keras.utils.to_categorical(y_train, num_classes)
    y_test_onehot = tf.keras.utils.to_categorical(y_test, num_classes)
    
    # Sınıf ağırlıklarını hesapla
    class_counts = np.bincount(y_train)
    total = len(y_train)
    class_weights = {i: np.sqrt(total / (num_classes * count)) for i, count in enumerate(class_counts)}
    
    print(f"Sınıf ağırlıkları: {class_weights}")
    print(f"Eğitim seti: {len(X_train)}, Test seti: {len(X_test)}")
    
    # BELLEK TEMİZLİĞİ
    del df
    gc.collect()
    
    return X_train, X_test, y_train_onehot, y_test_onehot, class_weights, label_encoder, num_classes, y_test

# BELLEK OPTİMİZASYONU: Generator kullan
def data_generator(texts, labels, tokenizer, batch_size, max_length):
    """Batch batch veri üretir - bellek dostu"""
    num_samples = len(texts)
    indices = np.arange(num_samples)
    
    while True:
        np.random.shuffle(indices)
        
        for start_idx in range(0, num_samples, batch_size):
            end_idx = min(start_idx + batch_size, num_samples)
            batch_indices = indices[start_idx:end_idx]
            
            batch_texts = [texts.iloc[i] for i in batch_indices]
            batch_labels = labels[batch_indices]
            
            # Tokenize
            encodings = tokenizer(
                batch_texts,
                max_length=max_length,
                padding='max_length',
                truncation=True,
                return_tensors='tf'
            )
            
            yield (encodings['input_ids'], encodings['attention_mask']), batch_labels

# Custom BERT Layer
class TFBertLayer(tf.keras.layers.Layer):
    def __init__(self, bert_model_name, **kwargs):
        super().__init__(**kwargs)
        self.bert_model_name = bert_model_name
        self.bert = TFAutoModel.from_pretrained(bert_model_name)
        self.bert.trainable = False  # BELLEK OPTİMİZASYONU: BERT'i dondur
        self.hidden_size = self.bert.config.hidden_size
    
    def call(self, inputs):
        input_ids, attention_mask = inputs
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        return outputs.pooler_output
    
    def compute_output_shape(self, input_shape):
        return (input_shape[0][0], self.hidden_size)
    
    def get_config(self):
        config = super().get_config()
        config.update({"bert_model_name": self.bert_model_name})
        return config

# BELLEK OPTİMİZASYONU: Daha küçük model
def create_bert_functional_model(num_classes, bert_model_name="dbmdz/bert-base-turkish-uncased", 
                                max_length=128):
    """Functional API ile BERT modeli"""
    print(f"Functional BERT modeli oluşturuluyor: {bert_model_name}")
    
    # Tokenizer
    tokenizer = AutoTokenizer.from_pretrained(bert_model_name)
    
    # Input layers
    input_ids = tf.keras.layers.Input(shape=(max_length,), dtype=tf.int32, name='input_ids')
    attention_mask = tf.keras.layers.Input(shape=(max_length,), dtype=tf.int32, name='attention_mask')
    
    # Custom BERT layer
    bert_output = TFBertLayer(bert_model_name, name='bert_layer')([input_ids, attention_mask])
    
    # BELLEK OPTİMİZASYONU: Daha küçük katmanlar
    x = tf.keras.layers.Dropout(0.3)(bert_output)
    x = tf.keras.layers.Dense(128, activation='relu')(x)  # 256->128
    x = tf.keras.layers.Dropout(0.2)(x)
    # İkinci dense layer'ı kaldır
    outputs = tf.keras.layers.Dense(num_classes, activation='softmax', dtype='float32')(x)
    
    model = tf.keras.Model(inputs=[input_ids, attention_mask], outputs=outputs)
    
    return model, tokenizer

def create_tf_dataset_simple(texts, labels, tokenizer, batch_size, max_length, shuffle=True, num_classes=9):
    """Generator kullanarak bellek dostu dataset - FIXED"""
    
    class TextGenerator:
        def __init__(self, texts, labels, tokenizer, max_length, num_classes):
            self.texts = texts
            self.labels = labels  # Bu zaten one-hot encoded
            self.tokenizer = tokenizer
            self.max_length = max_length
            self.num_classes = num_classes
        
        def __len__(self):
            return len(self.texts)
        
        def __call__(self):
            for i in range(len(self.texts)):
                text = str(self.texts.iloc[i])
                label = self.labels[i]  # Zaten one-hot encoded (shape: (9,))
                
                # Sadece bir örnek tokenize et
                encoding = self.tokenizer(
                    text,
                    max_length=self.max_length,
                    padding='max_length',
                    truncation=True,
                    return_tensors='tf'
                )
                
                # One-hot encoding YAPMA - zaten one-hot!
                # label artık direkt kullanılabilir
                
                yield {
                    'input_ids': encoding['input_ids'][0],
                    'attention_mask': encoding['attention_mask'][0]
                }, label  # Direkt label kullan (zaten one-hot)
    
    print(f"Generator dataset oluşturuluyor... ({len(texts)} örnek)")
    
    generator = TextGenerator(texts, labels, tokenizer, max_length, num_classes)
    
    # Output signature tanımla - ONE-HOT LABELS
    output_signature = (
        {
            'input_ids': tf.TensorSpec(shape=(max_length,), dtype=tf.int32),
            'attention_mask': tf.TensorSpec(shape=(max_length,), dtype=tf.int32)
        },
        tf.TensorSpec(shape=(num_classes,), dtype=tf.float32)  # One-hot labels
    )
    
    dataset = tf.data.Dataset.from_generator(
        generator,
        output_signature=output_signature
    )
    
    if shuffle:
        dataset = dataset.shuffle(buffer_size=min(1000, len(texts)))
    
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(tf.data.AUTOTUNE)
    
    # Output formatını düzenle
    def reformat(features, labels):
        return (features['input_ids'], features['attention_mask']), labels
    
    dataset = dataset.map(reformat)
    
    print(f"Generator dataset hazır: {len(texts)} örnek, batch_size={batch_size}")
    return dataset

def plot_confusion_matrix(y_true, y_pred, classes, model_name, normalize=False, cmap=plt.cm.Blues):
    """
    Confusion matrix çizer ve kaydeder
    """
    cm = confusion_matrix(y_true, y_pred)
    
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        title = 'Normalized Confusion Matrix'
    else:
        title = 'Confusion Matrix'
    
    plt.figure(figsize=(12, 10))
    sns.heatmap(cm, annot=True, fmt='.2f' if normalize else 'd', cmap=cmap,
                xticklabels=classes, yticklabels=classes, cbar_kws={'shrink': 0.8})
    
    plt.title(f'{title} - {model_name}', fontsize=16, fontweight='bold')
    plt.ylabel('True Label', fontsize=12)
    plt.xlabel('Predicted Label', fontsize=12)
    plt.xticks(rotation=45, ha='right')
    plt.yticks(rotation=0)
    
    # Kaydet
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f"outputs/confusion_matrices/{model_name}_confusion_matrix_{timestamp}.png"
    plt.tight_layout()
    plt.savefig(filename, dpi=300, bbox_inches='tight')
    plt.close()
    
    print(f"Confusion matrix kaydedildi: {filename}")
    return cm

def calculate_macro_metrics(y_true, y_pred, label_encoder, model_name):
    """
    Macro precision, recall, f1-score ve accuracy hesaplar
    """
    # Metrikleri hesapla
    accuracy = accuracy_score(y_true, y_pred)
    precision_macro = precision_score(y_true, y_pred, average='macro', zero_division=0)
    recall_macro = recall_score(y_true, y_pred, average='macro', zero_division=0)
    f1_macro = f1_score(y_true, y_pred, average='macro', zero_division=0)
    
    # Detaylı classification report
    class_report = classification_report(y_true, y_pred, 
                                        target_names=label_encoder.classes_,
                                        zero_division=0)
    
    # Sonuçları yazdır
    print("\n" + "="*60)
    print("MACRO METRİKLER - DETAYLI SONUÇLAR")
    print("="*60)
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision (Macro): {precision_macro:.4f}")
    print(f"Recall (Macro): {recall_macro:.4f}")
    print(f"F1-Score (Macro): {f1_macro:.4f}")
    print("\nClassification Report:")
    print(class_report)
    
    # CSV dosyasına kaydet
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    metrics_filename = f"outputs/{model_name}_macro_metrics_{timestamp}.csv"
    
    metrics_data = {
        'Model': [model_name],
        'Timestamp': [timestamp],
        'Accuracy': [accuracy],
        'Precision_Macro': [precision_macro],
        'Recall_Macro': [recall_macro],
        'F1_Score_Macro': [f1_macro]
    }
    
    metrics_df = pd.DataFrame(metrics_data)
    metrics_df.to_csv(metrics_filename, index=False, encoding='utf-8')
    print(f"\nMacro metrikler kaydedildi: {metrics_filename}")
    
    # Classification report'u txt dosyasına kaydet
    report_filename = f"outputs/{model_name}_classification_report_{timestamp}.txt"
    with open(report_filename, 'w', encoding='utf-8') as f:
        f.write(f"Model: {model_name}\n")
        f.write(f"Timestamp: {timestamp}\n")
        f.write("="*50 + "\n")
        f.write("MACRO METRİKLER\n")
        f.write("="*50 + "\n")
        f.write(f"Accuracy: {accuracy:.4f}\n")
        f.write(f"Precision (Macro): {precision_macro:.4f}\n")
        f.write(f"Recall (Macro): {recall_macro:.4f}\n")
        f.write(f"F1-Score (Macro): {f1_macro:.4f}\n\n")
        f.write("DETAYLI CLASSIFICATION REPORT:\n")
        f.write("="*50 + "\n")
        f.write(class_report)
    
    print(f"Classification report kaydedildi: {report_filename}")
    
    return {
        'accuracy': accuracy,
        'precision_macro': precision_macro,
        'recall_macro': recall_macro,
        'f1_macro': f1_macro
    }

def evaluate_model_with_metrics(model, X_test, y_test, tokenizer, label_encoder, model_name, batch_size=8, max_length=128):
    """
    Modeli değerlendirir ve confusion matrix + macro metrikleri hesaplar
    """
    print("Model değerlendirmesi başlıyor...")
    
    # Test dataset oluştur
    test_dataset = create_tf_dataset_simple(X_test, y_test, tokenizer, batch_size, max_length, shuffle=False)
    
    # Tahminleri yap
    print("Tahminler yapılıyor...")
    y_pred_proba = model.predict(test_dataset, verbose=1)
    y_pred = np.argmax(y_pred_proba, axis=1)
    
    # Gerçek etiketleri al (one-hot'den normal forma çevir)
    y_true = np.argmax(y_test, axis=1)
    
    # Confusion matrix çiz
    print("Confusion matrix oluşturuluyor...")
    cm = plot_confusion_matrix(y_true, y_pred, label_encoder.classes_, model_name)
    
    # Normalized confusion matrix
    cm_normalized = plot_confusion_matrix(y_true, y_pred, label_encoder.classes_, model_name, normalize=True)
    
    # Macro metrikleri hesapla
    print("Macro metrikler hesaplanıyor...")
    macro_metrics = calculate_macro_metrics(y_true, y_pred, label_encoder, model_name)
    
    # Bellek temizliği
    del test_dataset
    gc.collect()
    
    return macro_metrics, cm, y_pred

# Model eğitim fonksiyonu - BELLEK OPTİMİZASYONLU
def train_model(model, X_train, y_train, X_test, y_test, tokenizer, class_weights, 
                model_name, max_length=128, batch_size=8, label_encoder=None, y_test_original=None):  # batch_size 16->8
    """Modeli eğitir"""
    
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    model_filename = f"{model_name}_{timestamp}"
    
    checkpoint_path = f"outputs/models/{model_filename}.keras"
    
    # BELLEK OPTİMİZASYONU: tf.data.Dataset kullan
    print("Training dataset oluşturuluyor...")
    train_dataset = create_tf_dataset_simple(X_train, y_train, tokenizer, batch_size, max_length, shuffle=True)
    
    print("Validation dataset oluşturuluyor...")
    val_dataset = create_tf_dataset_simple(X_test, y_test, tokenizer, batch_size, max_length, shuffle=False)
    
    # Steps hesapla
    steps_per_epoch = len(X_train) // batch_size
    validation_steps = len(X_test) // batch_size
    
    # Callbacks
    checkpoint = ModelCheckpoint(
        checkpoint_path,
        monitor='val_loss',  # f1_score yerine loss
        verbose=1,
        save_best_only=True,
        mode='min'
    )
    
    early_stop = EarlyStopping(
        monitor='val_loss',
        patience=2,  # 3->2
        verbose=1,
        restore_best_weights=True
    )
    
    # BELLEK OPTİMİZASYONU: Garbage collection callback
    class GarbageCollectionCallback(tf.keras.callbacks.Callback):
        def on_epoch_end(self, epoch, logs=None):
            gc.collect()
            tf.keras.backend.clear_session()
    
    print(f"Model eğitimi başlıyor: {model_name}")
    
    model_start_time = time.time()
    history = model.fit(
        train_dataset,
        steps_per_epoch=steps_per_epoch,
        epochs=2,  # 10->5 (test için)
        validation_data=val_dataset,
        validation_steps=validation_steps,
        callbacks=[checkpoint, early_stop, GarbageCollectionCallback()],
        verbose=1
    )
    model_end_time = time.time()
    model_train_time = model_end_time - model_start_time
    
    history_df = pd.DataFrame(history.history)
    history_path = f"outputs/history/{model_filename}_history.csv"
    history_df.to_csv(history_path, index=False)
    print(f"Eğitim geçmişi kaydedildi: {history_path}")
    
    # Model değerlendirme
    print("Model değerlendiriliyor...")
    model_start_time = time.time()
    evaluation = model.evaluate(val_dataset, steps=validation_steps, verbose=1)
    model_end_time = time.time()
    model_test_time = model_end_time - model_start_time

    evaluation_results = dict(zip(model.metrics_names, evaluation))
    evaluation_results["Train Time"] = model_train_time
    evaluation_results["Test Time"] = model_test_time
    
    print(f"Değerlendirme sonuçları: {evaluation_results}")
    
    # Confusion matrix ve macro metrikleri hesapla
    if label_encoder is not None:
        print("\nConfusion Matrix ve Macro Metrikler hesaplanıyor...")
        macro_metrics, cm, y_pred = evaluate_model_with_metrics(
            model, X_test, y_test, tokenizer, label_encoder, model_name, 
            batch_size=batch_size, max_length=max_length
        )
        
        # Macro metrikleri evaluation_results'a ekle
        evaluation_results.update(macro_metrics)
    
    # Bellek temizliği
    del train_dataset, val_dataset
    gc.collect()
    
    return history, evaluation_results

def visualize_metrics(history, model_name):
    """Eğitim metriklerini görselleştirir"""
    metrics_dict = history.history if hasattr(history, 'history') else history
    
    metrics = [
        ('loss', 'Model Loss'),
        ('accuracy', 'Accuracy'),
    ]
    
    print("Metrikler görselleştiriliyor...")
    
    for metric, title in metrics:
        if metric in metrics_dict:
            plt.figure(figsize=(10, 6))
            plt.plot(metrics_dict[metric], label=f'Train {title}')
            val_metric = f'val_{metric}'
            if val_metric in metrics_dict:
                plt.plot(metrics_dict[val_metric], label=f'Validation {title}')
            plt.title(title)
            plt.xlabel('Epoch')
            plt.ylabel(title)
            plt.legend()
            plt.grid(True)
            plt.savefig(f"outputs/images/{model_name}_{metric}.png")
            plt.close()
    
    print(f"Metrik görselleştirmeleri kaydedildi")

def save_results_to_csv(model_name, eval_results):
    """Model sonuçlarını CSV'ye kaydeder"""
    results_path = "outputs/results.csv"
    
    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    
    result_data = {
        'model_name': model_name,
        'timestamp': timestamp,
        **eval_results
    }
    
    file_exists = os.path.isfile(results_path)
    
    with open(results_path, 'a', newline='', encoding='utf-8') as f:
        writer = csv.DictWriter(f, fieldnames=result_data.keys())
        
        if not file_exists:
            writer.writeheader()
            
        writer.writerow(result_data)
    
    print(f"Sonuçlar kaydedildi: {results_path}")

# Ana fonksiyon
def train_news_classification_model(data_path, stopwords_path=None, 
                                    model_name="Turkish_BERT_Classifier",
                                    max_length=128, batch_size=8, sample_size=None):
    """BERT tabanlı haber sınıflandırma - BELLEK OPTİMİZASYONLU"""
    
    create_directories()
    check_gpu()
    
    # Veriyi yükle - artık y_test_original da dönüyor
    X_train, X_test, y_train, y_test, class_weights, label_encoder, num_classes, y_test_original = \
        load_and_preprocess_data(data_path, stopwords_path=stopwords_path, 
                                max_length=max_length, sample_size=sample_size)
    
    # Model oluştur
    model, tokenizer = create_bert_functional_model(
        num_classes=num_classes,
        bert_model_name="dbmdz/bert-base-turkish-uncased",
        max_length=max_length
    )
    
    # Compile
    optimizer = tf.keras.optimizers.Adam(learning_rate=2e-5)
    
    model.compile(
        optimizer=optimizer,
        loss='categorical_crossentropy',
        metrics=['accuracy']  # Sadece accuracy
    )
    
    print(model.summary())
    
    # Eğit - artık label_encoder parametresi de veriliyor
    history, eval_results = train_model(
        model, X_train, y_train, X_test, y_test, tokenizer,
        class_weights, model_name, max_length=max_length, 
        batch_size=batch_size, label_encoder=label_encoder
    )
    
    visualize_metrics(history, model_name)
    save_results_to_csv(model_name, eval_results)
    
    # Belleği temizle
    tf.keras.backend.clear_session()
    gc.collect()
    
    print(f"\n{'='*50}")
    print(f"Model Eğitimi Tamamlandı!")
    print(f"Confusion Matrix ve Macro Metrikler kaydedildi!")
    print(f"{'='*50}\n")
    
    return model, history, eval_results, label_encoder, tokenizer

# KULLANIM
if __name__ == "__main__":
    try:
        # BELLEK OPTİMİZASYONU: Küçük parametreler
        model, history, eval_results, label_encoder, tokenizer = \
            train_news_classification_model(
                data_path="/kaggle/input/turkish-news-dataset/cleaned.csv",
                stopwords_path='/kaggle/input/turkish-news-dataset/stop-words.txt',
                model_name="BERT",
                max_length=128,      # 256->128
                batch_size=8,        # 16->8
            )
        
        print("Model başarıyla eğitildi!")
        print("Tüm metrikler ve confusion matrix kaydedildi!")
        
    except Exception as e:
        
        print(f"Hata oluştu: {str(e)}")
        import traceback
        print(traceback.format_exc())

In [3]:
import shutil
shutil.make_archive("bert", 'zip', "./outputs")
print("Sonuçlar 'bert.zip' olarak kaydedildi")

Sonuçlar 'bert.zip' olarak kaydedildi
